In [13]:
import csv
import pandas as pd
import requests
import json
import numpy as np
import sys
import os

In [14]:
sys.path.append("..")
from all_functions import *

## Combine jsons from AWS

In [39]:
path_to_json = '../OUTPUTS_languages'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]


In [40]:
all_languages_dict={}
for js in json_files:
    with open(os.path.join(path_to_json, js)) as json_file:
        new_json=json.load(json_file)
    all_languages_dict.update(new_json)

## Create dictionary of languages with wof:id

In [41]:
wof_ids = read_data("..\Analyzed_data\\final_titles_ids_all_status.csv")

In [42]:
mapping_dic_wof_name={}
for index, row in wof_ids.iterrows():
    key = row['wk:page']
    value = str(int(row['wof:id']))
    mapping_dic_wof_name.update({key:value})

In [54]:
mapping_dic_name_wof={}
for index, row in wof_ids.iterrows():
    key = str(int(row['wof:id']))
    value = row['wk:page']
    mapping_dic_name_wof.update({key:value})

In [43]:
dictionary_wof_languages={}
for key, value in all_languages_dict.iteritems():
    try:
        a = key.encode('utf8')
        new_key=mapping_dic_wof_name[a]
        dictionary_wof_languages.update({new_key:value})
    except KeyError:
        pass

## Discard names that were not cleaned up - see titles

In [46]:
import pickle
with open('..\Jupyter_notebooks_with_analysis\\names_do_not_want', 'rb') as f:
    names_do_not_want = pickle.load(f)

In [47]:
wof_do_not_want = []
for item in names_do_not_want:
    wof = int(item)
    wof_do_not_want.append(wof)

In [48]:
flattened = []
for x in wof_do_not_want:
    if type(x) is list:
        flattened.extend(x)
    else:
        flattened.append(x)

In [49]:
names_set=set(flattened)

In [50]:
dictionary_wof_languages_clean={}
for key, value in dictionary_wof_languages.iteritems():
    if key in names_set:
        pass
    else:
        dictionary_wof_languages_clean.update({key:value})

In [53]:
with open('..\Analyzed_data\\languages_dict_clean_wof.json', 'w') as outfile:
    json.dump(dictionary_wof_languages, outfile)

### Clean language names

In [62]:
dictionary_names_languages_clean={}
for key, value in dictionary_wof_languages_clean.iteritems():
    try:
        new_key=mapping_dic_name_wof[key]
        dictionary_names_languages_clean.update({new_key:value})
    except KeyError:
        pass

In [66]:
with open('..\Analyzed_data\\languages_dict_clean.json', 'w') as outfile:
    json.dump(dictionary_names_languages_clean, outfile)

### Clean languages from placetypes 

In [67]:
dic_wof_clean_all = {}
for key , values in dictionary_names_languages_clean.iteritems():
    dic_wof_values_clean = {}
    for lang, value in values.iteritems():
        if value.find(" (") != -1:
            lhs, rhs = value.split(" (",1)
            new_value = lhs
        elif value.find(",") != -1:
            lhs, rhs = value.split(",",1)
            new_value = lhs
        else:
            new_value = value
        dic_wof_values_clean.update({lang:new_value})
    dic_wof_clean_all.update({key:dic_wof_values_clean})

In [68]:
with open('..\Analyzed_data\\languages_wof_dict_clean_values.json', 'w') as outfile:
    json.dump(dic_wof_clean_all, outfile)